<a href="https://colab.research.google.com/github/spirosChv/neuro208/blob/main/practicals/Practical_3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulating dendrites - Part 2: Segregated dendrites

In this notebook we will see how segragated dendrites affect the propagation to the soma.

In [ ]:
!pip install neuron --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
from neuron.units import mV, ms, um
h.load_file("stdrun.hoc")

In [ ]:
# @title Make nicer plots -- Execute this cell
def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

In [ ]:
# Simulation parameters	
tstop = 100 * ms  # simulation time (ms)
h.dt = 0.1 * ms  # integration step (ms)
vinit = -65 * mV  # initial voltage (mV)

In [ ]:
# create five compartments using the command `h.Section()`
soma = h.Section(name='soma')
dend0, dend1, dend2, dend3 = [h.Section(name=n) for n in ['dend0', 'dend1', 'dend2', 'dend3']]

# Define properties of soma
soma.diam = 20 * um  # diameter (um)
soma.L = 20 * um  # length (um)
soma.Ra = 100  # Axial resistance (Ohm * cm)
soma.cm = 1  # specific membrane capacitance (uF/cm2)   
soma.nseg = 1  # number of segments

# Insert Hodgkin-Huxley channels (including leak)
soma.insert('hh')
for seg in soma: 
  seg.hh.gnabar = 0.05  # Sodium max conductance in (S/cm2)
  seg.hh.gkbar = 0.036  # Potassium max conductance in (S/cm2)
  seg.hh.gl = 0.0003  # Leak conductance (S/cm2)
  seg.hh.el = -65  # Reversal potential (mV)

# Define properties of dend0
dend0.diam = 5 * um  # diameter (um)
dend0.L = 500 * um  # length (um)
dend0.cm = 1  # specific membrane capacitance (uF/cm2) 
dend0.Ra = 100  # Axial resistance (Ohm * cm)
dend0.nseg = 11  # number of segments

dend0.insert('pas')  # Insert passive (leak) channels
for seg in dend0:
  seg.pas.e = -65  # Leak reversal potential (mV)
  seg.pas.g = 0.0003  # leak conductance (S/cm2)

# Define properties of dend1
dend1.diam = 3 * um  # diameter (um)
dend1.L = 200 * um  # length (um)
dend1.cm = 1  # specific membrane capacitance (uF/cm2)
dend1.Ra = 100  # Axial resistance (Ohm*cm)
dend1.nseg = 13  # number of segments

dend1.insert('pas')  # Insert passive (leak) channels
for seg in dend1:
  seg.pas.e = -65  # leak reversal potential (mV)
  seg.pas.g = 0.0003  # leak conductance (S/cm2)
    
# Define properties of dend0
dend2.diam = 5 * um  # diameter (um)
dend2.L = 500 * um  # length (um)
dend2.cm = 1  # specific membrane capacitance (uF/cm2) 
dend2.Ra = 100  # Axial resistance (Ohm * cm)
dend2.nseg = 11  # number of segments

dend2.insert('pas')  # Insert passive (leak) channels
for seg in dend2:
  seg.pas.e = -65  # leak reversal potential (mV)
  seg.pas.g = 0.0003  # leak conductance (S/cm2)

# Define properties of dend1
dend3.diam = 3 * um  # diameter (um)
dend3.L = 200 * um  # length (um)
dend3.cm = 1  # specific membrane capacitance (uF/cm2)
dend3.Ra = 100  # Axial resistance (Ohm*cm)
dend3.nseg = 13  # number of segments

dend3.insert('pas')  # Insert passive (leak) channels
for seg in dend3:
  seg.pas.e = -65  # leak reversal potential (mV)
  seg.pas.g = 0.0003  # leak conductance (S/cm2)

Let's now connect the compartments!

In [ ]:
# Connect the compartments together
# Connect the `0` point of `dend0` to the `0` point of the soma, i.e., soma(0)
dend0.connect(soma(0), 0)
# Connect the `0` point of dend1 to the `1` point of the dend0, i.e., dend0(1)
dend1.connect(dend0(1), 0)
# Connect the `0` point of `dend2` to the `0` point of the soma, i.e., soma(0)
dend2.connect(soma(1), 0)
# Connect the `0` point of dend3 to the `1` point of the dend2, i.e., dend2(1)
dend3.connect(dend2(1), 0)

Let's plot the neuron's shape

In [ ]:
h.topology()

In [ ]:
ps = h.PlotShape()
ps.plot(plt).mark(soma(0.5))
plt.show()

Notice that the soma is represented by the red dot.

The next step is to define the synapse(s), and the objects that connect a synapse with a pre-synaptic cell or event.

In [ ]:
# ====================== Synaptic stimulation ==================================    
# Place the synapse to the middle (0.5) of the dend1
syn0 = h.ExpSyn(dend1(0.5))
syn0.e = 0  # Reversal potential of the synapse (mV)

# Place the synapse to the middle (0.5) of the dend3
syn1 = h.ExpSyn(dend3(0.5))
syn1.e = 0  # Reversal potential of the synapse (mV)

# ================ create an artificial spike ==================================
tsignal = 50
ns = h.NetStim(0.5)
ns.start = tsignal  # onset time (ms)
ns.number = 1  # number of pre-synaptic events

# ... and connect the event to the synapses.
nc0 = h.NetCon(ns, syn0)
nc0.delay = 0  # delay (ms)

nc1 = h.NetCon(ns, syn1)
nc1.delay = 0  # delay (ms)

### Task 1: We will add one `h.ExpSyn` in the middle of `dend1` and one in the middle of `dend3`. 
### Activate (`weight*3`) each one of them and both of them. Is the response the linear sum?

In [ ]:
nc0.weight[0] = 0.008*0
nc1.weight[0] = 0.008*3

# Calculate and print in terminal the depolarization at the soma
vsoma_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

print(f'\nSomatic depolarization is {np.round(vsoma_vec.max() - vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')

**Hint!**

Try different combinations of `nc0.weight[0]` and `nc1.weight[0]`

 - Activate `nc1` instead of `nc0`.
 - Activate both `nc0` and `nc1`.

Now, you can experiment with various combinations!

In [ ]:
nsyn1 = 0  # @param {type:"slider", min:0, max:18, step:1}
nsyn2 = 0  # @param {type:"slider", min:0, max:18, step:1}

nc0.weight[0] = 0.008*nsyn1
nc1.weight[0] = 0.008*nsyn2

# Calculate and print in terminal the depolarization at the soma
vsoma_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

# reinitialize the simulator and run again
h.finitialize(vinit)
h.continuerun(tstop)

# Remove the first 20ms to avoid artifacts
tremove = 20
vsoma_vec.remove(0, int(tremove/h.dt))
t_vec.remove(0, int(tremove/h.dt))

plt.figure(figsize=(8, 6))
plt.plot(t_vec, vsoma_vec, color='black', label='soma')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

print(f'\nSomatic depolarization is {np.round(vsoma_vec.max() - vsoma_vec[int((tsignal-tremove)/h.dt)-1], 2)} mV')